**<h1>Setup and data processing<h1>**

---



<h3>Package installation<h3>


In [ ]:
%pip install pandas
%pip install python-binance
%pip install ta
%pip install matplotlib

<h3>Imports<h3>

In [ ]:
import pandas as pd
from binance.client import Client
import ta
import matplotlib.pyplot as plt

<h3>Coin Specs<h3>

In [ ]:
symbol = "BTCUSDT"
timeframe = "1h"
starting_date = "01 january 2017"

<h3>Binance data<h3>

In [ ]:
# download the coin info
info = Client().get_historical_klines(symbol, timeframe, starting_date)

In [ ]:
# storing it into a pandas data frame
dl_data = pd.DataFrame(info, columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_av', 'trades', 'tb_base_av', 'tb_quote_av', 'ignore' ])
dl_data

In [ ]:
# copy of the dowloaded data 
data = dl_data.copy()

In [ ]:
# keeping only the necessary columns
data.drop(columns=data.columns.difference(['timestamp','open','close','high','low','volume']), inplace=True)
data

In [ ]:
# formating the index
data.set_index(data['timestamp'], inplace=True)
data.index = pd.to_datetime(data.index, unit='ms')
del data['timestamp']
data

In [ ]:
type(data['open'][1])

In [ ]:
# formating the numbers
data["close"] = pd.to_numeric(data["close"])
data["high"] = pd.to_numeric(data["high"])
data["low"] = pd.to_numeric(data["low"])
data["open"] = pd.to_numeric(data["open"])

In [ ]:
type(data['open'][1])

<h3>Indicators<h3>

In [ ]:
data['MA-st'] = ta.trend.sma_indicator(data['close'], 200)
data['MA-lt'] = ta.trend.sma_indicator(data['close'], 800)
# data['MA-st'] = ta.trend.ema_indicator(data['close'], 200)
# data['MA-lt'] = ta.trend.ema_indicator(data['close'], 800)

data

In [ ]:
data.dropna(inplace=True)
data

**<h1>Backtest<h1>**

---

<h3>Wallet<h3>

In [ ]:
starting_usdt = 1000
starting_coin = 0
trading_fees = 0.001

<h3>Buy-Sell condition functions<h3>

In [ ]:
def buy_condition(row):
    if row['MA-st'] > row['MA-lt']:
        return True
    else:
        return False
    
#     return row['MA-st'] > row['MA-lt']:


def sell_condition(row):
    if row['MA-st'] < row['MA-lt']:
        return True
    else:
        return False

<h3>Backtest loop<h3>

In [ ]:
usdt = starting_usdt
coin = starting_coin
trades = []
wallet = []
buyhold = []
for index, row in data.iterrows():

    value = row['close']

    if buy_condition(row) and usdt > 0:
        coin = usdt / value
        coin = coin * (1 - trading_fees)
#         coin *= 1 - trade_fees
        usdt = 0
        trades.append({'date':index, 'side':'buy', 'price':value, 'coin':coin, 'usdt':usdt, 'wallet':coin*value})
        print(f"Bought {symbol} at {value}$ on the {index}")

    if sell_condition(row) and coin > 0: 
        usdt = coin *  value
        usdt = usdt * (1 - trading_fees)
        coin = 0
        trades.append({'date':index, 'side':'sell', 'price':value, 'coin':coin, 'usdt':usdt, 'wallet':usdt})
        print(f"Sold {symbol} at {value}$ on the {index}")

    if usdt == 0:
        wallet.append(coin * value)
    else:
        wallet.append(usdt)

    buyhold.append(starting_usdt / data["close"].iloc[0] * value)


In [ ]:
trades = pd.DataFrame(trades, columns = ['date', 'side', 'price', 'coin', 'usdt', 'wallet'])
trades = trades.round(2) #round numbers
trades


In [ ]:
trades['side'].value_counts()

**<h1>Results<h1>**


---



In [ ]:
print(
    "\nStarting amount: %s USDT"
    % (
        "{:0,.2f}".format(starting_usdt),

    )
)
print(
    "Buy-Hold: \t %s USDT\t (%s%% profit)"
    % (
        "{:0,.2f}".format(buyhold[-1]),
        "{:0,.2f}".format((buyhold[-1] / starting_usdt - 1) * 100),
    )
)
print(
    "2-SMA: \t\t %s USDT\t (%s%% profit)"
    % (
        "{:0,.2f}".format(wallet[-1]),
        "{:0,.2f}".format((wallet[-1] / starting_usdt - 1) * 100),
    )
)

plt.figure(figsize=(10, 6))
plt.plot(
    data.index,
    wallet,
    label="Wallet",
    color="gold",
)
plt.plot(
    data.index,
    buyhold,
    label="Buy-Hold",
    color="purple",
)
plt.legend(fontsize=18, loc="upper left")
plt.ylabel("Value [usdt]", fontsize=20)
plt.yticks(fontsize=14)
plt.xlabel("Date", fontsize=20)
plt.xticks(fontsize=14)
plt.tight_layout()